# Military Power Clustering

Source : globalfirepower.com on 1st may 2020

In [151]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import Libraries

In [152]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, StandardScaler, PowerTransformer, MinMaxScaler, RobustScaler
from sklearn.model_selection import KFold, cross_val_predict, train_test_split, GridSearchCV, cross_val_score, cross_validate
from sklearn.linear_model import LinearRegression, Lasso, Ridge,ElasticNet
from sklearn.metrics import plot_confusion_matrix, r2_score, mean_absolute_error, mean_squared_error, classification_report, confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import make_scorer, precision_score, precision_recall_curve, plot_precision_recall_curve, plot_roc_curve, roc_auc_score, roc_curve, f1_score, accuracy_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, ExtraTreesRegressor, AdaBoostClassifier
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_classif, f_regression, mutual_info_regression
from xgboost import XGBRegressor, XGBClassifier
from xgboost import plot_importance
from sklearn.pipeline import Pipeline
from sklearn.tree import plot_tree
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import warnings
warnings.filterwarnings('ignore')
warnings.warn("this will not show")
plt.rcParams["figure.figsize"] = (10,6)
pd.set_option('max_colwidth',200)

# pd.set_option('display.max_rows', 100) # if you wish to see more rows rather than default, just uncomment this line.
pd.set_option('display.max_columns', 200)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import colorama
from colorama import Fore, Style  # maakes strings colored
# !pip3 install termcolor
from termcolor import colored

# User Defined Function

In [153]:
# Function for determining the number and percentages of missing values

def missing (df):
    missing_number = df.isnull().sum().sort_values(ascending=False)
    missing_percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Missing_Number', 'Missing_Percent'])
    return missing_values

In [154]:
# To view summary information about the column

def first_looking(col):
    print("column name    : ", col)
    print("--------------------------------")
    print("per_of_nulls   : ", "%", round(df[col].isnull().sum()/df.shape[0]*100, 2))
    print("num_of_nulls   : ", df[col].isnull().sum())
    print("num_of_uniques : ", df[col].nunique())
    print(df[col].value_counts(dropna = False))

# Ingest Data  

In [197]:
df0 = pd.read_csv("../input/world-military-power-2020/World military power.csv", header=1)
df = df0.copy()

In [198]:
# df0 = pd.read_csv("World military power.csv")
# df = df0.copy()

In [199]:
df0.head(1)

# EDA

## Implement basic steps to see how is your data looks like

In [200]:
df.head(3)

In [201]:
df.tail(3)

In [202]:
df.sample(10)

In [203]:
df.columns

In [204]:
df.columns = df.columns.str.lower().str.replace('&', '_').str.replace(' ', '_')

In [205]:
df.columns

In [206]:
df.shape

In [207]:
print("There is", df.shape[0], "observation and", df.shape[1], "columns in the dataset")

In [208]:
df.info()

In [209]:
df.describe().T

In [210]:
df.describe(include=object).T

In [211]:
df.nunique()

In [212]:
# to find how many unique values object features have
for col in df.select_dtypes(include=[np.number]).columns:
  print(f"{col} has {df[col].nunique()} unique value")

In [213]:
df.duplicated().value_counts()

In [214]:
duplicate = df[df.duplicated()]

In [215]:
duplicate

In [216]:
df.drop_duplicates(keep=False, inplace=True)

In [217]:
df.shape

## Convert multi index columns to one level

In [218]:
df.head(1)

In [219]:
df.rename(columns={'military_strength': 'country'}, inplace=True)

In [220]:
df.head(1)

## Detect and drop duplicated columns

In [221]:
df.columns

In [222]:
drop_columns = ['aircraft_strength',
               'fighter/interceptor_strength',
               'attack_aircraft_strength',
               'transport_aircraft_fleet_strength',
               'trainer_aircraft_fleet',
               'helicopter_fleet_strength',
               'attack_helicopter_fleet_strength',
               'tank_strength',
               'afv/apc_strength',
               'self-propelled_artillery_strength',
               'towed_artillery_strength',
               'rocket_projector_strength',
               'navy_fleet_strengths',
               'aircraft_carrier_fleet_strength',
               'submarine_fleet_strength',
               'destroyer_fleet_strength',
               'frigate_fleet_strength',
               'defense_spending_budget',
               'external_debt',
               'airport_totals', 
               'oil_production',
               'oil_consumption',
               'proven_oil_reserves',
               'available_manpower',
               'total_population',
               'total_square_land_area',
               'total_coastline_coverage',
               'total_waterway_coverage',
               'total_border_coverage',
               ]

In [223]:
df.drop(drop_columns, axis = 1, inplace = True)

In [224]:
df.head(1)

In [225]:
df.shape

## Check for missing values 

In [226]:
missing (df)

## Deal with missing values 

In [227]:
# submarine_fleet_strength_value : South Korea --> 22
# frigate_fleet_strength_value : South Korea --> 4
# destroyer_fleet_strength_value : South Korea --> 12

In [228]:
df.loc[(df['country']=="South Korea") & (df['submarine_fleet_strength_value'].isnull()), 'submarine_fleet_strength_value'] = 22

In [229]:
df.loc[(df['country']=="South Korea") & (df['frigate_fleet_strength_value'].isnull()), 'frigate_fleet_strength_value'] = 4

In [230]:
df.loc[(df['country']=="South Korea") & (df['destroyer_fleet_strength_value'].isnull()), 'destroyer_fleet_strength_value'] = 12

In [231]:
missing (df)

In [232]:
df.fillna(0, inplace=True)

In [233]:
missing (df)

## Fix the dtypes

In [234]:
df.dtypes

In [235]:
df.head(1)

In [236]:
df = df.reset_index()   # df = df.reset_index(drop = True)
del df['index']

In [237]:
df.set_index('country', inplace=True)

In [238]:
for col in df.columns:
    if df.dtypes[col] == np.object:
        df[col] = df[col].apply(lambda x : x.replace(",", "") if type(x) != int else x).astype("float")

In [239]:
df.info()

In [240]:
missing(df)

## Drop the features that not affect military power

In [241]:
df.head(1)

In [242]:
df.columns

In [243]:
df.shape

In [244]:
for col in df.select_dtypes(include=[np.number]).columns:
    if df[col].min() == 0:
        print(col)

In [245]:
# Georeferences

# "total_coastline_coverage_value"
# "total_waterway_coverage_value"
# "total_border_coverage_value"
# "total_square_land_area_value"

In [246]:
df[df["total_coastline_coverage_value"]==0]["total_coastline_coverage_value"]

In [247]:
df[df["total_waterway_coverage_value"]==0]["total_waterway_coverage_value"]

In [248]:
df[df["total_border_coverage_value"]==0]["total_border_coverage_value"]

In [249]:
df[df["total_square_land_area_value"]==0]["total_square_land_area_value"]

In [250]:
drop_columns = ["total_coastline_coverage_value",
               "total_waterway_coverage_value",
               "total_border_coverage_value",
               "total_square_land_area_value"]

In [251]:
df.drop(drop_columns, axis = 1, inplace = True)

In [252]:
df.shape

In [253]:
df.columns

## Visualy inspect the some features

In [254]:
df.head(1)

In [255]:
sns.pairplot(df, palette="inferno", corner=True);

In [256]:
for col in df.columns:
    plt.figure(figsize = (22,6))
    sns.barplot(y = df[col], x = df.index, data = df)
    plt.xticks(rotation = 45);

## Descriptive statistics

In [259]:
df.describe().T

# Data Preprocessing

In [270]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, scale
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.cluster import KMeans

In [261]:
df1 = df.copy()

In [262]:
df1.reset_index(inplace=True)

In [263]:
df1.head(1)

In [264]:
X = df1.drop(["country"], axis = 1)

In [265]:
scaler = scale

In [266]:
X_scaled = pd.DataFrame(scaler(X))

In [267]:
X_scaled.head(1)

In [268]:
X_scaled.shape

# Modelling

In [271]:
K_means_model = KMeans(random_state = 101)

In [272]:
K_means_model.fit_predict(X)

## Hopkins Test 
#### Assess the clusterability of a dataset. A score between 0 and 1, a score around 0.5 express no clusterability, a score tending to 1 express an uniform distrubution so clustering can't be useful and  0 express  not an uniform distrubution hence clusterering can be useful.

In [275]:
!pip install pyclustertend

In [276]:
from pyclustertend import hopkins

In [277]:
hopkins(df, df.shape[0])

In [278]:
df11 = df.copy()
df12 = df.copy()
df13 = df.copy()

In [279]:
hopkins(scale(df11), df11.shape[0])

In [280]:
hopkins(MinMaxScaler().fit_transform(df12), df12.shape[0])

In [281]:
hopkins(StandardScaler().fit_transform(df13), df13.shape[0])

In [282]:
hopkins(scale(df), df.shape[0])

## Clustering with K-means

### Elbow Method 
We use this method to find optimal k value by looking up to break point of graph. 

In [283]:
ssd = []

K = range(2,10)

for k in K:
    model = KMeans(n_clusters =k, random_state=101)
    model.fit(X_scaled)
    ssd.append(model.inertia_)

In [284]:
plt.plot(K, ssd, "bo-")
plt.xlabel("Different k values")
plt.ylabel("inertia-error") 
plt.title("elbow method") 

In [285]:
ssd # sum of squared distance

In [286]:
pd.Series(ssd).diff()

In [287]:
df1_diff =pd.DataFrame(-pd.Series(ssd).diff()).rename(index = lambda x : x+1)
df1_diff

In [288]:
df1_diff.plot(kind='bar')

In [289]:
from yellowbrick.cluster import KElbowVisualizer

model_ = KMeans(random_state=101)
visualizer = KElbowVisualizer(model_, k=(2,9))

visualizer.fit(X_scaled)        # Fit the data to the visualizer
visualizer.show();

### Silhouette analysis

In [290]:
from sklearn.metrics import silhouette_score

In [291]:
silhouette_score(X_scaled, K_means_model.labels_)

In [292]:
range_n_clusters = range(2,9)
for num_clusters in range_n_clusters:
    # intialise kmeans
    kmeans = KMeans(n_clusters=num_clusters, random_state=101)
    kmeans.fit(X_scaled)
    cluster_labels = kmeans.labels_
    # silhouette score
    silhouette_avg = silhouette_score(X_scaled, cluster_labels)
    print(f"For n_clusters={num_clusters}, the silhouette score is {silhouette_avg}")

In [293]:
from sklearn.cluster import KMeans

from yellowbrick.cluster import SilhouetteVisualizer

model3 = KMeans(n_clusters=4, random_state=101)
visualizer = SilhouetteVisualizer(model3)

visualizer.fit(X_scaled)    # Fit the data to the visualizer
visualizer.poof();

### Model Building and label visualisation

In [294]:
model = KMeans(n_clusters = 4, random_state = 101)
model.fit_predict(X_scaled)

In [295]:
model.labels_

In [296]:
X["clusters"] = model.fit_predict(X_scaled)

In [297]:
X.head()

In [298]:
X.clusters.value_counts()

In [299]:
df1.head(1)

In [300]:
df1["predicted_clusters"] = model.fit_predict(X_scaled)

In [305]:
df1["predicted_clusters"].value_counts().plot(kind="pie", startangle=0, labeldistance=1.1, autopct='%1.1f%%',figsize=(10,10));

In [306]:
df1.predicted_clusters.value_counts()

sns.countplot(x=df1["predicted_clusters"], data=df1)
for index,value in enumerate(df1.predicted_clusters.value_counts().sort_index()):
     plt.text(index, value, f"{value}", ha="center", va="bottom", fontsize = 13)

In [307]:
df1[df1["predicted_clusters"]==0][["country", "military_strength_power_index"]].sort_values(by="military_strength_power_index")

In [308]:
df1[df1["predicted_clusters"]==1][["country", "military_strength_power_index"]].sort_values(by="military_strength_power_index")

In [309]:
df1[df1["predicted_clusters"]==2][["country", "military_strength_power_index"]].sort_values(by="military_strength_power_index")

In [310]:
df1[df1["predicted_clusters"]==3][["country", "military_strength_power_index"]].sort_values(by="military_strength_power_index")

### Evaluate 

In [311]:
df1['military_strength_power_index'].describe()

In [312]:
df1['military_strength_power_index'].value_counts().index.sort_values()

In [313]:
df1['military_strength_power_index'].iplot(kind='hist', bins=200)

In [335]:
def classification(military_strength_power_index):
    if military_strength_power_index <= 0.065:
        return 1
    elif 0.065 < military_strength_power_index <= 0.1:
        return 2
    elif 0.1 < military_strength_power_index <= 0.5:
        return 3
    else:
        return 0

In [336]:
df1['classification'] = df1['military_strength_power_index'].apply(classification)

In [337]:
df1.head(1)

In [338]:
df1['classification'].value_counts()

In [339]:
df1["classification"].value_counts().plot(kind="pie", startangle=0, labeldistance=1.1, autopct='%1.1f%%',figsize=(10,10));

In [340]:
df1.classification.value_counts()

sns.countplot(x=df1["classification"], data=df1)
for index,value in enumerate(df1.classification.value_counts().sort_index()):
     plt.text(index, value, f"{value}", ha="center", va="bottom", fontsize = 13)

In [341]:
df1[df1["classification"]==2][["country", "military_strength_power_index"]].sort_values(by="military_strength_power_index")

In [342]:
len(df1[df1["classification"]==2]["country"])

In [343]:
df1[df1["classification"]==1][["country", "military_strength_power_index"]].sort_values(by="military_strength_power_index")

In [344]:
len(df1[df1["classification"]==1]["country"])

In [345]:
df1[df1["classification"]==0][["country", "military_strength_power_index"]].sort_values(by="military_strength_power_index")

In [346]:
len(df1[df1["classification"]==0]["country"])

In [347]:
df1[df1["classification"]==3][["country", "military_strength_power_index"]].sort_values(by="military_strength_power_index")

In [348]:
len(df1[df1["classification"]==3]["country"])

In [349]:
from sklearn.metrics import adjusted_rand_score

In [350]:
df1_end = df1
df1_end

In [351]:
df1.predicted_clusters.value_counts()

In [352]:
df1.classification.value_counts()

In [353]:
df1[["classification", "predicted_clusters"]].value_counts()

In [354]:
df1[["country", "classification", "predicted_clusters"]]

In [355]:
df1[df1["classification"]!=df1["predicted_clusters"]][["country", "classification", "predicted_clusters"]]

## Hierarchical Clustering

In [356]:
df2 = df.copy()

In [357]:
df2.head(1)

In [358]:
X_scaled = scale(df2)

In [359]:
pd.DataFrame(X_scaled)

### Hopkins Test

In [360]:
hopkins(df2, df2.shape[0])

In [362]:
df21 = df.copy()
df22 = df.copy()
df23 = df.copy()

In [363]:
hopkins(scale(df21), df21.shape[0])

In [364]:
hopkins(MinMaxScaler().fit_transform(df22), df22.shape[0])

In [365]:
hopkins(StandardScaler().fit_transform(df23), df23.shape[0])

In [366]:
hopkins(scale(df2), df.shape[0])

### Dendogram

In [367]:
from scipy.cluster.hierarchy import dendrogram, linkage

In [368]:
hc_ward = linkage(y = df2, method = "ward")
hc_complete = linkage(df2, "complete")
hc_average = linkage(df2, "average")
hc_single = linkage(df2, "single")

In [369]:
plt.figure(figsize = (14,7))
dendrogram(hc_ward, leaf_font_size = 10);

In [370]:
plt.figure(figsize = (14,7))
dendrogram(hc_ward,
           truncate_mode = "lastp",
           p = 10,
           show_contracted = True,
           leaf_font_size = 10);

In [371]:
plt.figure(figsize = (20,12))

plt.subplot(221)
plt.title("Ward")
plt.xlabel("Observations")
plt.ylabel("Distance")
dendrogram(hc_ward, truncate_mode = "lastp", p = 10, show_contracted = True, leaf_font_size = 10)

plt.subplot(222)
plt.title("Complete")
plt.xlabel("Observations")
plt.ylabel("Distance")
dendrogram(hc_complete, truncate_mode = "lastp", p = 10, show_contracted = True, leaf_font_size = 10)

plt.subplot(223)
plt.title("Average")
plt.xlabel("Observations")
plt.ylabel("Distance")
dendrogram(hc_average, truncate_mode = "lastp", p = 10, show_contracted = True, leaf_font_size = 10)

plt.subplot(224)
plt.title("Single")
plt.xlabel("Observations")
plt.ylabel("Distance")
dendrogram(hc_single, truncate_mode = "lastp", p = 10, show_contracted = True, leaf_font_size = 10);

In [372]:
sns.clustermap(df2);

In [373]:
sns.clustermap(df2, col_cluster=False);

In [374]:
sns.clustermap(df2, row_cluster=False);

### Model Building and label visualisation

In [375]:
from sklearn.cluster import AgglomerativeClustering

### S(Silhouette) Score

In [376]:
from sklearn.metrics import silhouette_score

In [377]:
K = range(2,10)

for k in K:
    model_labels = AgglomerativeClustering(n_clusters = k,  
                                    affinity='euclidean', 
                                    linkage='ward').fit_predict(X_scaled)
    print(f'Silhouette Score for {k} clusters: {silhouette_score(X_scaled, model_labels)}')

In [378]:
model = AgglomerativeClustering(n_clusters = 5,
                                affinity='euclidean',
                                linkage='ward')

In [379]:
clusters = model.fit_predict(X_scaled)

In [380]:
plt.scatter(df2.iloc[:,0], df2.iloc[:,1], c = clusters, cmap = "viridis")

In [382]:
plt.figure(figsize = (20,60))

plt.subplot(9,3,1)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,1], c = clusters, cmap = "viridis");
plt.subplot(9,3,2)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,2], c = clusters, cmap = "viridis");
plt.subplot(9,3,3)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,3], c = clusters, cmap = "viridis");
plt.subplot(9,3,4)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,4], c = clusters, cmap = "viridis");
plt.subplot(9,3,5)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,5], c = clusters, cmap = "viridis");
plt.subplot(9,3,6)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,6], c = clusters, cmap = "viridis");
plt.subplot(9,3,7)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,7], c = clusters, cmap = "viridis");
plt.subplot(9,3,8)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,8], c = clusters, cmap = "viridis");
plt.subplot(9,3,9)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,9], c = clusters, cmap = "viridis");
plt.subplot(9,3,10)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,10], c = clusters, cmap = "viridis");
plt.subplot(9,3,11)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,11], c = clusters, cmap = "viridis");
plt.subplot(9,3,12)
plt.title("Original")
plt.scatter(df2.iloc[:,0], df2.iloc[:,12], c = clusters, cmap = "viridis");

In [383]:
df2["predicted_clusters_hc"] = clusters

In [384]:
df2.head()

In [385]:
df2.groupby("predicted_clusters_hc").mean()

In [386]:
df2.predicted_clusters_hc.value_counts()

In [387]:
df2[df2["predicted_clusters_hc"]==0].index

In [388]:
df2[df2["predicted_clusters_hc"]==1].index

In [389]:
df2[df2["predicted_clusters_hc"]==2].index

In [390]:
df2[df2["predicted_clusters_hc"]==3].index

In [391]:
df2[df2["predicted_clusters_hc"]==4].index

### Evaluate 

In [405]:
df2["military_power"] = df2["predicted_clusters_hc"].map({  0 : "low",
                                                            1 : "medium",
                                                            2 : "most_powerfull",
                                                            3 : "powerfull",
                                                            4 : "weak",
                                                         })

In [406]:
df2.head(1)

In [407]:
df2.reset_index(inplace=True)

In [408]:
df2["predicted_clusters_hc"].value_counts().plot(kind="pie", startangle=0, labeldistance=1.1, autopct='%1.1f%%',figsize=(10,10));

In [409]:
df2.predicted_clusters_hc.value_counts()

In [410]:
df2.predicted_clusters_hc.value_counts()

sns.countplot(x=df2["predicted_clusters_hc"], data=df2)
for index,value in enumerate(df2.predicted_clusters_hc.value_counts().sort_index()):
     plt.text(index, value, f"{value}", ha="center", va="bottom", fontsize = 13)

In [411]:
df2.groupby("predicted_clusters_hc")["country"].unique()

In [412]:
df2["military_power"].value_counts()

In [414]:
df2.groupby("military_power")["country"].unique()["weak"]

In [415]:
df2.groupby("military_power")["country"].unique()["low"]

In [416]:
df2.groupby("military_power")["country"].unique()["medium"]

In [417]:
df2.groupby("military_power")["country"].unique()["powerfull"]

In [418]:
df2.groupby("military_power")["country"].unique()["most_powerfull"]

In [419]:
df1['military_strength_power_index'].iplot(kind='hist', bins=200)

In [420]:
def classification1(military_strength_power_index):
    if military_strength_power_index <= 0.065:
        return "most_powerfull"
    elif 0.065 < military_strength_power_index <= 0.070:
        return "powerfull"
    elif 0.070 < military_strength_power_index <= 0.1:
        return "medium"
    elif 0.1 < military_strength_power_index <= 0.85:
        return "low"
    else:
        return "weak"

In [421]:
df.military_strength_power_index.sort_values()

In [422]:
df2['classification'] = df2['military_strength_power_index'].apply(classification1)

In [423]:
df2['classification'].value_counts()

In [424]:
df2['military_power'].value_counts()

In [425]:
df2["classification"].value_counts().plot(kind="pie", startangle=0, labeldistance=1.1, autopct='%1.1f%%',figsize=(10,10));

In [426]:
df2.classification.value_counts()

In [427]:
df2.classification.value_counts()

sns.countplot(x=df2["classification"], data=df2)
for index,value in enumerate(df2.classification.value_counts()):
     plt.text(index, value, f"{value}", ha="center", va="bottom", fontsize = 13)

In [428]:
df2[["classification", "military_power"]].value_counts()

In [429]:
df2[["country", "classification", "military_power"]]

In [430]:
df2[df2["classification"]!=df2["military_power"]][["country", "classification", "military_power"]]